In [13]:

from __future__ import print_function
from six.moves import xrange
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

from dataclasses import dataclass

@dataclass(eq=True,frozen=True)
class Point:
    x: float
    y: float

###########################
# Problem Data Definition #
###########################
def create_data_model():
    """Stores the data for the problem"""
    data = {}
    # Locations in block unit

    input_data = open("../tsp/data/tsp_51_1").read()
    lines = input_data.split('\n')
    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
    _locations = list(map(lambda x:(x.x,x.y),points))
    # Compute locations in meters using the block dimension defined as follow
    # Manhattan average block: 750ft x 264ft -> 228m x 80m
    # here we use: 114m x 80m city block
    # src: https://nyti.ms/2GDoRIe "NY Times: Know Your distance"
    data["locations"] = _locations
    data["num_locations"] = len(data["locations"])
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data


#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
    """Computes the Manhattan distance between two points"""
#     return (abs(position_1[0] - position_2[0]) +
#             abs(position_1[1] - position_2[1]))
    return ((position_1[0] - position_2[0]) * (position_1[0] - position_2[0]) +
            (position_1[1] - position_2[1]) * (position_1[1] - position_2[1]))

def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = {}
    # precompute distance between location to have distance callback in O(1)
    for from_node in xrange(data["num_locations"]):
        _distances[from_node] = {}
        for to_node in xrange(data["num_locations"]):
            if from_node == to_node:
                _distances[from_node][to_node] = 0
            else:
                _distances[from_node][to_node] = (manhattan_distance(
                    data["locations"][from_node], data["locations"][to_node]))

    def distance_evaluator(from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return _distances[from_node][to_node]

    return distance_evaluator


###########
# Printer #
###########
def print_solution(routing, assignment):
    """Prints assignment on console"""
    print('Objective: {}'.format(assignment.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(routing.IndexToNode(index))
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(routing.IndexToNode(index))
    plan_output += 'Distance of the route: {}m\n'.format(distance)
    print(plan_output)


########
# Main #
########
def main():
    """Entry point of the program"""
    # Instantiate the data problem.
    data = create_data_model()

    # Create Routing Model
    routing = pywrapcp.RoutingModel(data["num_locations"],
                                    data["num_vehicles"], data["depot"])
    # Define weight of each edge
    distance_evaluator = create_distance_evaluator(data)
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator)

    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)
    print_solution(routing, assignment)


main()

Objective: 4186
Route:
 0 -> 32 -> 17 -> 49 -> 48 -> 22 -> 1 -> 20 -> 37 -> 21 -> 25 -> 31 -> 39 -> 50 -> 43 -> 29 -> 38 -> 15 -> 14 -> 44 -> 16 -> 18 -> 42 -> 11 -> 40 -> 19 -> 7 -> 13 -> 35 -> 4 -> 8 -> 46 -> 3 -> 41 -> 24 -> 34 -> 23 -> 30 -> 12 -> 36 -> 6 -> 26 -> 47 -> 27 -> 45 -> 9 -> 10 -> 28 -> 2 -> 5 -> 33 -> 0
Distance of the route: 4186m

